In [1]:
# https://www.kaggle.com/wimwim/wavenet-lstm

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import glob

import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow import keras
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import optimizers, callbacks, layers
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, \
    Conv1D, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D, Input, Multiply, LSTM
from tensorflow.keras.models import Model, Sequential, load_model

from utils import MultiHeadSelfAttention

import random

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mape(y_true, y_pred):
    return 100*np.mean((np.abs(y_true-y_pred)/y_true))

def mish(x):
    return x*K.tanh(K.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 25
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))    

es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


In [4]:
def wave_block(filters,kernel_size,n):
    def f(x):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters=filters,
                    kernel_size=1, 
                    padding='same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters=filters,
                    kernel_size=kernel_size, 
                    padding='same',
                    activation = 'tanh',
                    dilation_rate=dilation_rate)(x)
            sigm_out = Conv1D(filters=filters,
                    kernel_size=kernel_size, 
                    padding='same',
                    activation = 'sigmoid',
                    dilation_rate=dilation_rate)(x)
            x = Multiply()([tanh_out,sigm_out])
            x = Conv1D(filters = filters,
                           kernel_size = 1,
                           padding='same')(x)
            res_x = Add()([res_x,x])
        return res_x
    return f

def wavenet(input_size):
    fs = 16
    ks = 3
    activation = mish
    input_ = Input(shape=(input_size, 1))
    
    x = wave_block(fs, 3,12)(input_)
    x = AveragePooling1D(2)(x)

    x = wave_block(fs*4, 3, 8)(x)
    x = AveragePooling1D(2)(x)
    
    x = wave_block(fs*4, 3, 4)(x)
    x = AveragePooling1D(2)(x)
    
    x= LSTM(fs*4, return_sequences=True, return_state=False)(x)
    x = MultiHeadSelfAttention(64)(x) # multihead attention from transformer
    
#     if adpative versiob of attention
#      x= LSTM(fs*4, return_sequences=True, return_state=True)(x)
#     x = tf.keras.layers.Attention()([x, h]) # addaptive version of attention

    x = GlobalAveragePooling1D()(x)
    
    output_ = Dense(30, activation='softmax')(x)
        
    model = Model(input_, output_)
    return model

In [5]:
m = wavenet(7*24)

In [6]:
m.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 168, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 168, 16)      32          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 168, 16)      784         conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 168, 16)      784         conv1d[0][0]                     
_______________________________________________________________________________________

In [7]:
from tensorflow.keras.utils import to_categorical

x = np.random.normal(0, 1, (1000, 7*24, 1))
y = np.random.randint(0, 30, 1000)
y = to_categorical(y)

In [8]:
m.compile(loss='categorical_crossentropy', optimizer='adam')

m.fit(x, y,
     epochs=20)

Epoch 1/20
32/32 [==============================] - 3s 81ms/step - loss: 3.4091
Epoch 2/20
32/32 [==============================] - 2s 59ms/step - loss: 3.4015
Epoch 3/20
32/32 [==============================] - 2s 58ms/step - loss: 3.3984
Epoch 4/20
32/32 [==============================] - 2s 58ms/step - loss: 3.3865
Epoch 5/20
32/32 [==============================] - 2s 58ms/step - loss: 3.3885
Epoch 6/20
32/32 [==============================] - 2s 59ms/step - loss: 3.3840
Epoch 7/20
32/32 [==============================] - 2s 58ms/step - loss: 3.3890
Epoch 8/20
32/32 [==============================] - 2s 56ms/step - loss: 3.3867
Epoch 9/20
32/32 [==============================] - 2s 57ms/step - loss: 3.3812
Epoch 10/20
32/32 [==============================] - 2s 60ms/step - loss: 3.3703
Epoch 11/20
32/32 [==============================] - 2s 57ms/step - loss: 3.3621
Epoch 12/20
32/32 [==============================] - 2s 57ms/step - loss: 3.3521
Epoch 13/20
32/32 [==================